# Configurando e criando a API do ISIC para obter acesso as imagens

In [ ]:
import requests


class ISICApi(object):
    def __init__(self, hostname='https://isic-archive.com',
                 username=None, password=None):
        self.baseUrl = f'{hostname}/api/v1'
        self.authToken = None

        if username is not None:
            if password is None:
                password = input(f'Password for user "{username}":')
            self.authToken = self._login(username, password)

    def _makeUrl(self, endpoint):
        return f'{self.baseUrl}/{endpoint}'

    def _login(self, username, password):
        authResponse = requests.get(
            self._makeUrl('user/authentication'),
            auth=(username, password)
        )
        if not authResponse.ok:
            raise Exception(f'Login error: {authResponse.json()["message"]}')

        authToken = authResponse.json()['authToken']['token']
        return authToken

    def get(self, endpoint):
        url = self._makeUrl(endpoint)
        headers = {'Girder-Token': self.authToken} if self.authToken else None
        return requests.get(url, headers=headers)

    def getJson(self, endpoint):
        return self.get(endpoint).json()

    def getJsonList(self, endpoint):
        endpoint += '&' if '?' in endpoint else '?'
        LIMIT = 50
        offset = 0
        while True:
            resp = self.get(
                f'{endpoint}limit={LIMIT:d}&offset={offset:d}'
            ).json()
            if not resp:
                break
            for elem in resp:
                yield elem
            offset += LIMIT

# Log do usuário

In [ ]:
import pandas as pd

api = ISICApi(username="lucarmo", password="abc123")
studyList = api.getJson('study?limit=50&state=complete')

df = pd.DataFrame(studyList)
df

,_id,description,name,updated
0,573f11119fc3c132505c0ee7,,ISBI 2016: 100 Lesion Classification,2016-11-08T10:25:45.485000+00:00
1,5a32cde91165975cf58a469c,,ISIC Annotation Study - All Features,2017-12-14T19:15:53.898000+00:00


In [ ]:
import pandas as pd
import os

# Conectando com a pasta do Google Drive

In [ ]:
os.chdir('drive/My Drive/')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Criando e salvando o arquivo em csv com os metadados das imagens

In [ ]:
# Initialize the API; no login is necessary for public data
api = ISICApi()
outputFileName = 'imagedata'

imageList = api.getJson('image?limit=23906&offset=0&sort=name')

print('Fetching metadata for %s images' % len(imageList))
imageDetails = []
for image in imageList:
    print(' ', image['name'])
    # Fetch the full image details
    imageDetail = api.getJson('image/%s' % image['_id'])
    imageDetails.append(imageDetail)

# Determine the union of all image metadata fields
metadataFields = set(
    field
    for imageDetail in imageDetails
    for field in imageDetail['meta']['clinical'].keys()
)
metadataFields = ['isic_id'] + sorted(metadataFields)

# Write the metadata to a CSV
print('Writing metadata to CSV: %s' % outputFileName+'.csv')
with open('TCC/TCC.csv', 'w') as outputStream:
    csvWriter = csv.DictWriter(outputStream, metadataFields)
    csvWriter.writeheader()
    for imageDetail in imageDetails:
        rowDict = imageDetail['meta']['clinical'].copy()
        rowDict['isic_id'] = imageDetail['name']
        csvWriter.writerow(rowDict)

# Lendo o conteúdo do arquivo csv e passando os dados para uma variável

In [ ]:
df = pd.read_csv('TCC.csv')
len(df)

23906

In [ ]:
column_names = ['isic_id','age_approx','anatom_site_general','benign_malignant','clin_size_long_diam_mm','diagnosis','diagnosis_confirm_type','family_hx_mm','mel_class','mel_thick_mm','melanocytic','nevus_type','personal_hx_mm','sex']
df_malignant = pd.DataFrame(columns = column_names)
df_malignant

,isic_id,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,diagnosis,diagnosis_confirm_type,family_hx_mm,mel_class,mel_thick_mm,melanocytic,nevus_type,personal_hx_mm,sex


In [ ]:
for index, row in df.iterrows():
    if row[3] == 'malignant':
        df_malignant = df_malignant.append(row)
df_malignant

,isic_id,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,diagnosis,diagnosis_confirm_type,family_hx_mm,mel_class,mel_thick_mm,melanocytic,nevus_type,personal_hx_mm,sex
2,ISIC_0000002,60.0,upper extremity,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,True,NaN,NaN,female
4,ISIC_0000004,80.0,posterior torso,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,True,NaN,NaN,male
13,ISIC_0000013,30.0,posterior torso,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,True,NaN,NaN,female
22,ISIC_0000022,55.0,lower extremity,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,True,NaN,NaN,female
26,ISIC_0000026,30.0,lower extremity,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,True,NaN,NaN,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23878,ISIC_0034294,70.0,posterior torso,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,True,NaN,NaN,female
23897,ISIC_0034313,55.0,lower extremity,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,True,NaN,NaN,male
23900,ISIC_0034316,85.0,upper extremity,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,True,NaN,NaN,male
23901,ISIC_0034317,70.0,lower extremity,malignant,NaN,melanoma,histopathology,NaN,NaN,NaN,True,NaN,NaN,female


In [ ]:
column_names = ['isic_id','age_approx','anatom_site_general','benign_malignant','clin_size_long_diam_mm','diagnosis','diagnosis_confirm_type','family_hx_mm','mel_class','mel_thick_mm','melanocytic','nevus_type','personal_hx_mm','sex']
df_benign = pd.DataFrame(columns = column_names)
df_benign

,isic_id,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,diagnosis,diagnosis_confirm_type,family_hx_mm,mel_class,mel_thick_mm,melanocytic,nevus_type,personal_hx_mm,sex


In [ ]:
for index, row in df.iterrows():
    if row[3] == 'benign':
        df_benign = df_benign.append(row)
df_benign

,isic_id,age_approx,anatom_site_general,benign_malignant,clin_size_long_diam_mm,diagnosis,diagnosis_confirm_type,family_hx_mm,mel_class,mel_thick_mm,melanocytic,nevus_type,personal_hx_mm,sex
0,ISIC_0000000,55.0,anterior torso,benign,NaN,nevus,NaN,NaN,NaN,NaN,True,NaN,NaN,female
1,ISIC_0000001,30.0,anterior torso,benign,NaN,nevus,NaN,NaN,NaN,NaN,True,NaN,NaN,female
3,ISIC_0000003,30.0,upper extremity,benign,NaN,nevus,NaN,NaN,NaN,NaN,True,NaN,NaN,male
5,ISIC_0000005,40.0,anterior torso,benign,NaN,nevus,NaN,NaN,NaN,NaN,True,NaN,NaN,female
6,ISIC_0000006,25.0,posterior torso,benign,NaN,nevus,NaN,NaN,NaN,NaN,True,NaN,NaN,female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23895,ISIC_0034311,55.0,posterior torso,benign,NaN,nevus,histopathology,NaN,NaN,NaN,True,NaN,NaN,male
23896,ISIC_0034312,40.0,NaN,benign,NaN,nevus,single image expert consensus,NaN,NaN,NaN,True,NaN,NaN,female
23898,ISIC_0034314,50.0,anterior torso,benign,NaN,nevus,histopathology,NaN,NaN,NaN,True,NaN,NaN,male
23903,ISIC_0034319,30.0,NaN,benign,NaN,nevus,histopathology,NaN,NaN,NaN,True,NaN,NaN,male


# Fazendo o download das imagens e separando em pastas benignas e malignas

In [ ]:
import urllib
import os

imageList = api.getJson('image?limit=23906&offset=0&sort=name')

print('Downloading %s images' % len(imageList))
imageDetails = []
for image in imageList:
    imageFileResp = api.get('image/%s/download' % image['_id'])
    imageFileResp.raise_for_status()
    
    if image['name'] in df_malignant.values:
        print(image['name'])
        imageFileOutputPath = os.path.join('maligno', '%s.jpg' % image['name'])
        with open(imageFileOutputPath, 'wb') as imageFileOutputStream:
            for chunk in imageFileResp:
                imageFileOutputStream.write(chunk)
                
    if image['name'] in df_benign.values:
        print(image['name'])
        imageFileOutputPath = os.path.join('benigno', '%s.jpg' % image['name'])
        with open(imageFileOutputPath, 'wb') as imageFileOutputStream:
            for chunk in imageFileResp:
                imageFileOutputStream.write(chunk)

A saída de streaming foi truncada nas últimas 5000 linhas.
ISIC_0027963
ISIC_0027964
ISIC_0027965
ISIC_0027968
ISIC_0027970
ISIC_0027971
ISIC_0027972
ISIC_0027974
ISIC_0027975
ISIC_0027977
ISIC_0027978
ISIC_0027979
ISIC_0027980
ISIC_0027981
ISIC_0027984
ISIC_0027985
ISIC_0027986
ISIC_0027987
ISIC_0027988
ISIC_0027992
ISIC_0027993
ISIC_0027994
ISIC_0027995
ISIC_0027997
ISIC_0027999
ISIC_0028000
ISIC_0028001
ISIC_0028002
ISIC_0028003
ISIC_0028004
ISIC_0028005
ISIC_0028006
ISIC_0028007
ISIC_0028008
ISIC_0028009
ISIC_0028010
ISIC_0028011
ISIC_0028012
ISIC_0028013
ISIC_0028014
ISIC_0028015
ISIC_0028016
ISIC_0028017
ISIC_0028018
ISIC_0028019
ISIC_0028021
ISIC_0028022
ISIC_0028023
ISIC_0028024
ISIC_0028025
ISIC_0028026
ISIC_0028027
ISIC_0028028
ISIC_0028029
ISIC_0028030
ISIC_0028031
ISIC_0028032
ISIC_0028033
ISIC_0028034
ISIC_0028036
ISIC_0028037
ISIC_0028038
ISIC_0028039
ISIC_0028040
ISIC_0028041
ISIC_0028042
ISIC_0028044
ISIC_0028045
ISIC_0028046
ISIC_0028047
ISIC_0028048
ISIC_0028049
ISIC_